In [2]:
import matplotlib
matplotlib.use('Agg')

#%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import numpy as np

from matplotlib import pyplot as plt
import scipy
from scipy.ndimage import gaussian_filter


import csv

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage as ndi

from skimage.segmentation import watershed
from skimage.feature import peak_local_max

import cv2
import os

import scipy.misc
#import skimage.filter

import parmap
import glob

import numpy as np
import matplotlib.pyplot as plt
import scipy.misc

from PIL import Image

    
clrs = ['red','blue','cyan','green','yellow','pink','magenta','black','lightgreen','lightblue']


Autosaving every 180 seconds


In [14]:

def grab_data_from_directories(root_dir, name, data):

    frames = []
    frame_ids = []
    locs_array = []
    tracks = []

    ctr = 0
    max_frames = 5000
    for k in range(data.shape[0]):
        track = data[k,0]
        frame_start = data[k,1]
        frame_end = data[k,2]
        n_frames = data[k, 3]
        print (track, frame_start ,frame_end, " # frames: ", n_frames , ' count: ', ctr)
        #     
        dir_ = os.path.join(root_dir,str(track))

        fnames = np.sort(glob.glob(dir_+"/*.npz"))


        for n in trange(frame_start, frame_end+1,1):
            res = np.int32([i.find('frame_'+str(n).zfill(7)) for i in fnames])
            idx = np.where(res>=0)[0]
            if idx.shape[0]==0:
                continue
            data2 = np.load(fnames[idx][0])

            frames.append(data2['frame'])
            frame_ids.append(data2['frame_id'])
            locs_array.append(data2['locs'])
            tracks.append(data2['track_id_sleap'])

            ctr+=1

            if ctr>=max_frames:
                break

        if ctr>=max_frames:
            break

    np.savez('/home/cat/'+name+'.npz',
             frames = frames,
             frame_ids = frame_ids,
             locs_array = locs_array,
             tracks = tracks
            )
    
def return_selected(data, start, end):
    
    track_ids = data[:,0]
    #print (track_ids)
    female = np.zeros((track_ids.shape[0],4),'float32')
    female[:,0] = track_ids
    female[:,1:3] = data[:,start:end]
    female[:,3] = data[:,end]
    
    idx = np.where(np.isnan(female))
    female[idx]=0
    female=np.int32(female)
    idx = np.where(female[:,1]==0)[0]
    female = np.delete(female, idx, axis=0)
    
    print ("# of examples: ", np.sum(female[:,3]))
    
    return female

# ROTATE IMAGE
def rotate_image(image, angle):
    image_center = tuple(np.array(image.shape[1::-1]) / 2)
    rot_mat = cv2.getRotationMatrix2D(image_center, angle, 1.0)
    result = cv2.warpAffine(image, rot_mat, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    return result

# FIND ROTATION 
def crop_frame_and_rotate(locs,
                  frame,
                  plotting,
                  rotation_flag,
                  rgb=False
                  ):  
    
    # This is the order that you should search for the centre of the animal; do not use median/mean; 
    centre_list = [7,6,5,8,4,9,3,10,2,11,1,12,0,13] # SPINE 2 - feautre 6 is centre of animal now;
    
    #
    padding_size = 400
    if rgb==True:
        frame_padded = np.zeros((1024+padding_size*2, 1280+padding_size*2, 3), 'uint8')
    else:
        frame_padded = np.zeros((1024+padding_size*2, 1280+padding_size*2), 'uint8')
        
    #print (frame_padded.shape, frame.shape)
    frame_padded[padding_size:padding_size+1024, padding_size:padding_size+1280] = frame
    #frame = np.pad(frame, 2, pad_with, padder=np.mean(frame))
    
    frame = frame_padded.copy()
    locs+=padding_size
    
    # how wide is the final image of the animal
    width = 100
    
    # temporary padding to help with rotation;
    padding = 150
       
    if False:
        centrex = int(np.nanmean(locs[:,1]))
        centrey = int(np.nanmean(locs[:,0]))
    else:
        # loop through all the feature from spine going outwards
        for k in range(len(centre_list)):   
            temp = locs[centre_list[k],0]
            if np.isnan(temp)==False:
                break

        centrey = int(locs[centre_list[k],0])
        centrex = int(locs[centre_list[k],1])
    
    # GRAB CENTRED PATCH
    frame_patch = frame[centrex-width-padding:centrex+width+padding, 
                        centrey-width-padding:centrey+width+padding]#[:,:,1] #GRAB SINGLE CHAN COLOR
    
    # ADD Pixels on top of the frame for the .png file
    if plotting:
        size = 6
        frame_image = frame_patch.copy()
        for k in range(locs.shape[0]):
            temp = locs[k,0]
            if np.isnan(temp)==True:
                continue
            tempy = int(locs[k,0])-centrey-padding-width
            tempx = int(locs[k,1])-centrex-padding-width
            frame_image[tempx-size: tempx+size,
                        tempy-size: tempy+size]= np.nan
    else:
        frame_image = None

    # EXCLUDE PATCHES THAT ARE NOT RIGHT SIZE DUE TO CLIPPING;
    # TODO: fix this so it is not required
    if frame_patch.shape[0]!= (width*2 + padding*2):
        return None
    
    if frame_patch.shape[1]!= (width*2 + padding*2):
        return None

    
    # if computing rotation
    if rotation_flag:

        # ROTATE RAW DATA USING A PCA ON THE FEATURES
        try:

            # FIND CENTRES
            idx = np.where(np.isnan(locs[:,0]))[0]
            locs = np.delete(locs, idx, 0)

            # FIND ROTATION USING PCA
            x = locs[:,0]
            y = locs[:,1]
            x = x - np.mean(x)
            y = y - np.mean(y)
            coords = np.vstack([x, y])
            if coords.shape[1]==1:
                frame_patch = frame_patch[padding:-padding, padding: -padding]
                return frame_patch, frame_image            

            cov = np.cov(coords)
            evals, evecs = np.linalg.eig(cov)

            sort_indices = np.argsort(evals)[::-1]
            x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
            x_v2, y_v2 = evecs[:, sort_indices[1]]
            theta = np.arctan((x_v1)/(y_v1))  

            # ROTATE IMAGE
            frame_rotated = rotate_image(frame_patch, -theta*180/3.14159)
            frame_rotated = frame_rotated[padding:-padding, padding: -padding]

            if plotting:
                # rotate png image (OPTIONAL)
                frame_image = rotate_image(frame_image, -theta*180/3.14159)
                frame_image = frame_image[padding:-padding, padding: -padding]

        except:
            #print ("Couldn't rotate image, returning original patch")
            #print ("coords = np.vstack([x, y]): ", coords, coords.shape)

            return frame_patch, frame_image

        return frame_rotated, frame_image

    # if no rotation requested just return images as is
    else:
        
        return frame_patch, frame_image


def try_float(v):
   try:
       return float(v)
   except Exception:
       return np.nan

def load_xy(trace):
    loc= np.array([try_float(item) for item in trace[2:]])
    locx = loc[::2]
    locy = loc[1::2]
    loc = np.stack((locx,locy)).T
    
    return loc


  
def generate_training_image_sleap(frame_ids,
                                   video_name,
                                   root_dir,
                                   plotting,
                                  traces,
                                  rotation_flag,
                                  rgb,
                                  specific_tracks,
                                  ):
    
    '''  This function extracts the images from the videos for each tracklet constructed by sleap
    
        Todo: save all the tracklet data in a single file rather than many files.
    
    '''
       
    # OPEN VIDEO
    original_vid = cv2.VideoCapture(video_name)

    # RESET VID TO FIRST FRAME ID (frame_ids are in sequence)
    start = frame_ids[0]
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)
    
    # FIND START FRAME IN SLEAP .CSV TRACES OUTPUT
    ctr_traces=0
    while True:
        frame_id2 =int(traces[ctr_traces][1])
        if frame_id2 >= start:
            break
        ctr_traces+=1
    
    if ctr_traces>=len(traces):
        ctr_traces=len(traces)-1
    
    # LOOP OVER ALL SELECTED FRAME_IDs
    for frame_id in frame_ids:
        ret, frame = original_vid.read()
        if rgb==False:
            green_chan = 1
            frame = frame[:,:,green_chan]
                
        # LOOP OVER ALL THE TRACKLETS IN THE FRAME
        while True:
            
            #if len(traces)<=ctr_traces:
            #    break
            try:
                frame_id2 =int(traces[ctr_traces][1])
            except:
                #print ("ctr_traces: ", ctr_traces, "  len(traces)", len(traces))
                #print ("frame ids first and last: ", frame_ids[0], frame_ids[-1])
                break
                
            track_id = int(traces[ctr_traces][0].replace('track_',''))
            
            if track_id in specific_tracks:
                pass
            else:
                ctr_traces+=1
                continue
                
    
            if  frame_id2 != frame_id:
                break
            
            # Generate save filename
            out_fname = root_dir+str(track_id)+'/frame_'+str(frame_id2).zfill(7)+"_id_"+str(track_id)+'.npy'

            
            #print ("track_id: ", track_id, "   frame: ", frame_ids[ctr_ids])
            loc = load_xy(traces[ctr_traces])

            ##############################################################
            ###################### GET X,Y LOCS ##########################
            ##############################################################
            tempx = loc[:,0]
            tempy = loc[:,1]
            locs = np.vstack((tempx,tempy)).T

            # exit if all np.nan vals
            #if locs.shape[0]==0:
            #    ctr_traces+=1
            #    continue

            ##############################################################
            ################### ROTATE IMG ###############################
            ##############################################################
            frame_rotated, frame_image = crop_frame_and_rotate(locs,
                                                               frame,
                                                               plotting,
                                                               rotation_flag,
                                                               rgb)

            ##############################################################
            ################### SAVE IMG AND NPY #########################
            ##############################################################
            # MAKE DIR IF NOT ALREADY MDE
            try:
                local_dir = root_dir+str(track_id)
                os.mkdir(local_dir)
            except:
                pass

            # SAVE IMAGE FOR MANUAL EVALUATION
            if plotting:
            
                #Rescale to 0-255 and convert to uint8
                rescaled = (255.0 / frame_image.max() * (frame_image - frame_image.min())).astype(np.uint8)

                im = Image.fromarray(rescaled)
                im.save(out_fname[:-4]+'.png')

            # SAVE DATA
            #if frame_rotated.shape[0]!=200:
            #    print (frame_id, track_id, frame_rotated.shape)
            np.savez(out_fname[:-4]+'.npz', 
                     frame=frame_rotated,
                     locs = locs,
                     frame_id = frame_id,
                     track_id_sleap = track_id)
            
            # MOVE TO NEXT VALUE IN SLEAP TRACES
            ctr_traces+=1
            
    
    # save stack of image data: - TODO
    
    
            
                       
def csv_to_traces(fname, n_animals):

    ''' Convert csv to the basic 2 files needed to generate traces for all animals
        1. traces_reassembled.npz contains tracesx, tracesy which correspond to the
        n animals selected. tracesx has dimensions [n_features*n_animals, n_frames]
        - For sleap output this is the .csv data reshaped from [n_detections,n_features]
          to [ ..., n_frames]
          filling in the list of animals whenever there is an animal detected in a particular
          frame. 

        2. chain_id.npy which has shape [n_frames, n_animals] and contains the track_id
        for each of the animal features sets in reassembled.
    '''

    import csv
    from tqdm import tqdm
    import numpy as np

    # LOAD .CSV DATA FIRST
    data_tracks = []
    with open(fname) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        ctr=0
        for row in tqdm(csv_reader):
            data_tracks.append([])
            for item in row:
                if item=='':
                    item = 1E10
                elif 'track' in item:
                    item = item.replace('track_','')
                data_tracks[ctr].append(item)
            ctr+=1
    data_tracks = np.array(data_tracks[1:])
    #print (data_tracks[0])

    # CONVERT DATA TO FLOAT32 and replace missing data with Nan
    #print (data_tracks.shape)
    data_tracks = np.float32(data_tracks)
    #print (data_tracks)

    idx = np.where(data_tracks==1E10)
    data_tracks[idx]=np.nan
    #print (data_tracks[0])
    #print ("Final tracks data shape: ", data_tracks.shape)
    
    
    #########################################################################
    ###### CONVERT data_tracks to inference and chain.npy files #############
    #########################################################################
    
    n_frames = int(np.max(data_tracks[:,1]))+1
    #print ("example data-tracks: ", data_tracks[240000:240002])
    #print ("Final tracks data shape: ", data_tracks.shape, 
    #       "  # frames: ", n_frames)


    # initialize array to hold track ids: [n_frames, track_id]
    #n_animals=4+4
    n_features=14
    traces_reassembled = np.zeros((n_frames,n_animals*n_features,2), 'float32')+np.nan
    #print ("traces reassembled: ", traces_reassembled.shape)

    # initizlied chain ids
    chain_ids = np.zeros((n_frames, n_animals))+np.nan
    #print ("chain_ids: ", chain_ids.shape)

    # placeholder for data
    sleap_locs = np.zeros((n_features, 2), 'float32')

    # loop over all tracks filling in the data in the 
    ctr = 0
    from tqdm import trange
    for k in trange(data_tracks.shape[0]):
        track_id = int(data_tracks[k,0])
        frame_id = int(data_tracks[k,1])
        #print ("index: ", k, " track_id: ", track_id,
        #       "  frame_id: ", frame_id, 
        #       " data: ", data_tracks[k])
        sleap_locs[:,0] = data_tracks[k,2:][::2]  # grab the x vals
        sleap_locs[:,1] = data_tracks[k,2:][1::2] # grab the y vals

        # find the next empty slot and enter the chain id info
        loc = 0
        while True:
            if np.isnan(chain_ids[frame_id,loc]):
                break
            loc+=1

        # insert the track_id in the chain
        chain_ids[frame_id,loc] = track_id

        # insert the feature locations 
        traces_reassembled[frame_id,
                           loc*n_features:(loc+1)*n_features] = sleap_locs

    # save data
    np.save(fname[:-4]+'_chain_ids.npy', chain_ids)
    np.save(fname[:-4]+'_traces_reassembled.npy', traces_reassembled)

    return chain_ids, traces_reassembled


def generate_CSUA_tracks_for_CNN_training_and_prediction(home_dir,
                                                     parallelize,
                                                     n_processes,
                                                     n_animals,
                                                     rotation_flag,
                                                     rgb,
                                                     specific_tracks,
                                                     plotting):

    root_dir = home_dir + '/cnn_output/'
    try:
        os.mkdir(root_dir)
    except:
        pass
    #plotting = True

    # LOAD VIDEO
    video_name = home_dir +'/'+os.path.split(home_dir)[1]+'.avi'
    print (video_name)

    # First find # frames and grab traces
    traces =[]
    fname_csv = video_name+'.predictions.analysis.csv'
    with open(fname_csv, mode='r') as infile:
        reader = csv.reader(infile)
        with open('coors_new.csv', mode='w') as outfile:
            writer = csv.writer(outfile)
            for row in reader:
                traces.append(row)

    # REMOVE HEADER
    traces.pop(0)
    print('# of lines of in leap track: ', len(traces))
    # SHOW FIRST LINE OF DATA

    # select all frames and make 
    cap = cv2.VideoCapture(video_name)
    total_frames = int(cap.get(7))
    print ("total # vid frames: ", total_frames)

    frame_ids =np.arange(total_frames)
    frame_ids_split = np.array_split(frame_ids, n_processes*10)

    if True:
        # LOAD ALL THE ANIMALS IN THAT FRAME AND SAVE THEM IN DIR
        if parallelize:
            parmap.map(generate_training_image_sleap,
                           frame_ids_split,
                           video_name,
                           root_dir,
                           plotting,
                           traces,
                           rotation_flag,
                           rgb,
                           specific_tracks,
                           pm_processes=n_processes,
                           pm_pbar=True)

        else:
            from tqdm import trange
            for k in trange(len(frame_ids_split)):
                generate_training_image_sleap(frame_ids_split[k],
                           video_name,
                           root_dir,
                           plotting,
                           traces,
                           rotation_flag,
                           rgb,
                          specific_tracks
                             )

    # After finishing, can run .csv -> traces -> chain_ids generation
    
    ##################################################################
    ####### .CSV TO traces_inferences.npy and chain_id.npy files #####
    ##################################################################
    # SELECT THE .CSV FILE
    chain_ids, traces_reassembled = csv_to_traces(fname_csv, n_animals)
    print ("chain_ids: ", chain_ids.shape, chain_ids[:5])
    print ("traces_reassembled: ", traces_reassembled.shape, traces_reassembled[:5])

        

In [16]:
############################################################################################
######### EXTRACT rotated images for CNN and Make traces.npy and chain_ids.npy files #######
############################################################################################
'''  Generate the .npz files needed for CNN classification.

    Prerequisite functions need to be run:
    - sleap-track (e.g. sleap_os_multifile.py function over many vids)
    - sleap-analysis (e.g. slp_to_h7_to_csv.py to convert data to .csv)

    Input: 
    1. directory of 1 hr video containing .csv and .avi files 

    Output:
    1. cnn_data/track #s directories containg the image (optional) and .npz file with the image
    2. traces_reassembled.npz file which has tracesx, tracesy which correspond to the
        n animals selected. tracesx has dimensions [n_features*n_animals, n_frames]
        - For sleap output this is the .csv data reshaped from [n_detections,n_features]
          to [ ..., n_frames]
          filling in the list of animals whenever there is an animal detected in a particular
          frame. 
    3. chain_id.npy which has shape [n_frames, n_animals] and contains the track_id
        for each of the animal features sets in reassembled.
    
'''

home_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed' 
parallelize = True
n_processes = 12
n_animals = 4
plotting = False
rotation_flag = False
rgb=True

# label only the specific tracks from the google spreadsheet;
specific_tracks = [749,827,1521,861,1732,411,1945,1623,89,1726,1756,1791,1599,1749,1792,1553,1785,1744]
         
generate_CSUA_tracks_for_CNN_training_and_prediction(home_dir,
                                                     parallelize,
                                                     n_processes,
                                                     n_animals,
                                                     rotation_flag,
                                                     rgb,
                                                     specific_tracks,
                                                     plotting)
                

/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/2020_3_15_11_53_51_617746_compressed.avi
# of lines of in leap track:  307652
total # vid frames:  89987


100%|██████████| 120/120 [06:43<00:00,  3.36s/it]
307653it [00:05, 59235.61it/s]
100%|██████████| 307652/307652 [00:04<00:00, 62167.30it/s]


chain_ids:  (89987, 4) [[ 0.  1. nan nan]
 [ 0.  1. nan nan]
 [ 0.  1. nan nan]
 [ 0.  1. nan nan]
 [ 0.  1.  2. nan]]
traces_reassembled:  (89987, 56, 2) [[[310.35138 572.4292 ]
  [      nan       nan]
  [328.01074 592.1255 ]
  [      nan       nan]
  [332.36166 620.2585 ]
  [322.48322 642.2069 ]
  [344.07208 678.09973]
  [352.18756 696.1015 ]
  [374.36697 704.43665]
  [432.44778 724.13495]
  [458.30075 742.2128 ]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [244.22726 656.8627 ]
  [278.35284 658.8743 ]
  [260.15955 672.67706]
  [261.95026 690.9479 ]
  [264.0267  712.6529 ]
  [280.0774  726.6587 ]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [216.22949 602.92316]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [      nan       nan]
  [  

# MAKE TRAINING DATA MANUALLY

In [17]:
##############################################################
################### MANUALLY MAKE TRAINING DATA ##############
##############################################################

print ("manualy create ground truth datasets")
print (" goto: https://docs.google.com/spreadsheets/d/1go6bELvzQd6O8jOkdmHGAvs3DdEdUsFMipGck63f5-M/edit#gid=0" )
print (' export as .csv and then follow instructions below')
print ('')


manualy create ground truth datasets
 goto: https://docs.google.com/spreadsheets/d/1go6bELvzQd6O8jOkdmHGAvs3DdEdUsFMipGck63f5-M/edit#gid=0

Next run 9_train_cnn (the notebook will gather the .npz files automatically for trainning)


In [18]:
##########################################################
############ READ INFO FROM GOOGLE SPREADSHEET ###########
##########################################################
fname = '/home/cat/Downloads/manual_id_grouping_2020_3_15_11_53_51_617746_compressed - Sheet.csv'

from numpy import genfromtxt
data = genfromtxt(fname, delimiter=',',dtype='float32')

# load 2nd and 3rd column for each group of data
female = return_selected(data, 1,3)
male = return_selected(data,4,6)    
pup1 = return_selected(data,7,9)
pup2 = return_selected(data,10,12)

print (female[:2])

# of examples:  10441
# of examples:  5057
# of examples:  5138
# of examples:  5982
[[  827 21204 21487   283]
 [  827 21497 22080   583]]


In [22]:
###################################################
####### LOAD TRACKS AND SAVE .NPZ FILES ###########
###################################################
from tqdm import trange
import json

root_dir = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020_3_15_11_53_51_617746_compressed/cnn_output/'

name = 'pup1'
data = pup1
    
grab_data_from_directories(root_dir, name, data)


  5%|▌         | 5/99 [00:00<00:01, 48.29it/s]

827 22596 22694  # frames:  98  count:  0


  4%|▍         | 7/175 [00:00<00:02, 63.42it/s]

1732 65600 65774  # frames:  174  count:  99


  3%|▎         | 5/166 [00:00<00:03, 48.66it/s]

...

  5%|▍         | 14/301 [00:00<00:02, 133.25it/s]

1744 66900 67200  # frames:  300  count:  4107


  2%|▏         | 14/730 [00:00<00:05, 132.80it/s]

1744 67231 67960  # frames:  729  count:  4408


 81%|████████  | 591/730 [00:05<00:01, 103.29it/s]


In [ ]:
######################################################################
############## MAKE CLASSIFIER TRAINING DATA FROM SAVED ##############
######################################################################
import numpy as np

root_animals = root_dir + '/training_images_cc/segments_training/'
names = ['female','male','pup1','pup2', 'block']

import glob

data = []
for ctr, name in enumerate(names):
    data.append([])

    fnames = glob.glob(root_animals+name+'/**/*',recursive = True)
    for fname in fnames:
        if '.npy' in fname:
            data[ctr].append(np.load(fname))

for k in range(len(names)):
    print (names[k], len(data[k]))
    temp = np.array(data[k])
    print (temp.shape)
    idx_rand = np.random.choice(np.arange(temp.shape[0]), 
                                #595, 
                                temp.shape[0]//2,
                                replace=False)
    print (k, idx_rand.shape)
    temp = np.array(temp[idx_rand])
    print (temp.shape)
    np.save(root_animals+"id_"+str(k)+'_train.npy', temp)
    
    # save test datasets; 
    temp = np.array(data[k])
    idx = np.arange(temp.shape[0])
    idx_test = np.delete(idx, idx_rand)
    print ("Idx test: ", idx_test.shape)
    
    temp = temp[idx_test]
    print ("saved test: ", temp.shape)
    np.save(root_animals+"id_"+str(k)+'_test.npy', temp)
    
    

female 1042
(1042, 200, 200)
0 (521,)
(521, 200, 200)
Idx test:  (521,)
saved test:  (521, 200, 200)
male 595
(595, 200, 200)
1 (297,)
(297, 200, 200)
Idx test:  (298,)
saved test:  (298, 200, 200)
pup1 1653
(1653, 200, 200)
2 (826,)
(826, 200, 200)
Idx test:  (827,)
saved test:  (827, 200, 200)
pup2 2449
(2449, 200, 200)
3 (1224,)
(1224, 200, 200)
Idx test:  (1225,)
saved test:  (1225, 200, 200)


(100, 100, 50)


In [252]:
# VISUALIZE MATCHES
clrs = ['red','blue','cyan','green','yellow','pink','magenta','white','lightgreen','lightblue']


for k in frames:  #skip first match
    print ("frame: ", k)
    im = surfaces_watershed[k]
    idx0 = np.where(im==0)
    #print ("idx0: ", idx0)
    im[idx0]==np.nan
    
    # 
    ids = np.unique(im)
    #print (ids)
    for ctr, id_ in enumerate(ids):
        if id_==0:
            continue

        #
        idx = np.where(im==id_)
        centrex = np.mean(idx[0])
        centrey = np.mean(idx[1])
        print (centrex.shape, centrex)
        
        #centrey = np.mean(im[idx[:,1]])

        match_id = np.array(matches[k][ctr]).squeeze()
        print ("matchid: ", match_id)
        
        plt.scatter(k, centrex+centrey, 
                    c=clrs[match_id%10])
        
plt.show()
        
        

frame:  0
() 20.92491467576792
matchid:  1
() 39.075
matchid:  3
() 62.41414141414141
matchid:  4
() 84.26804123711341
matchid:  5
frame:  1
() 17.873303167420815
matchid:  1
() 30.59722222222222
matchid:  1
() 39.14646464646464
matchid:  3
() 62.38693467336683
matchid:  4
() 84.29473684210527
matchid:  5
frame:  2
() 22.23017902813299
matchid:  1
() 38.775401069518715
matchid:  3
() 62.51269035532995
matchid:  4
() 84.2258064516129
matchid:  5
frame:  3
() 22.31782945736434
matchid:  1
() 38.776595744680854
matchid:  2
() 62.51269035532995
matchid:  4
() 84.23684210526316
matchid:  5
frame:  4
() 22.40776699029126
matchid:  1
() 38.71584699453552
matchid:  2
() 62.759803921568626
matchid:  3
() 84.14778325123153
matchid:  4
frame:  5
() 22.395734597156398
matchid:  1
() 38.73157894736842
matchid:  2
() 62.67464114832536
matchid:  4
() 84.09661835748793
matchid:  5
frame:  6
() 22.27972027972028
matchid:  1
() 39.20873786407767
matchid:  2
() 62.748768472906406
matchid:  4
() 84.317948

TypeError: unsupported operand type(s) for %: 'NoneType' and 'int'

In [215]:
id_ = 145
print (np.unique(surfaces_watershed[id_-1]))
print (np.unique(surfaces_watershed[id_]))
print (matches[id_])


[0 4 6 7]
[0 3 5 6]
[[0], [4], [6], [7]]


In [94]:
##################################################################
#################### EXTRACT PATCHES AND ALIGN THEM ##############
##################################################################

#import skimage.filter


fig=plt.figure()


#img = misc.imread('oval.png', flatten=1)

video_name ='/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'


original_vid = cv2.VideoCapture(video_name)

frame_id= 900
original_vid.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
ret, frame = original_vid.read()

img1 = img_array1[frame_id]

aspect = 'equal'
n_rows = np.unique(img1).shape[0]
for ctr_id, id_ in enumerate(np.unique(img1)):
    
    img = img_array1[frame_id].copy()
    
    #id_ = 5
    print ("ctr_ID: ", ctr_id)

    # PLOT FRAME
    ax=plt.subplot(n_rows,5,1+ctr_id*5)
    plt.imshow(frame)
    print (frame.shape)
    
    # PLOT FRAME 
    ax=plt.subplot(n_rows,5,2+ctr_id*5)
    plt.imshow(img, aspect=aspect)

    
    # ZERO OUT ALL NON ID_ values for PCA ROTATION COMPUTATION
    print (np.unique(img1))
    idx1 = np.where(img1==id_)
    idx2 = np.where(img1!=id_)
    img[idx1]=1
    img[idx2]=0


    width = 125
    padding = 30
    print ("idx1: ", idx1[0],
                    idx1[1])
    centrex = int(np.nanmean(idx1[0]))*8
    centrey = int(np.nanmean(idx1[1]))*8
    print ("cenres:", centrex, centrey,
          centrex//8, centrey//8)

    #frame_patch = np.zeros((200,200),'int32')
    frame_patch = frame[centrex-width-padding:centrex+width+padding, centrey-width-padding:centrey+width+padding][:,:,1]

    ax=plt.subplot(n_rows,5,3+ctr_id*5)
    plt.imshow(frame_patch,aspect=aspect)
    
    
    #
    y, x = np.nonzero(img)
    x = x - np.mean(x)
    y = y - np.mean(y)
    coords = np.vstack([x, y])

    cov = np.cov(coords)
    evals, evecs = np.linalg.eig(cov)

    sort_indices = np.argsort(evals)[::-1]
    x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
    x_v2, y_v2 = evecs[:, sort_indices[1]]


    # PLOT THE PATCH WITH ANIMAL IN IT
    ax=plt.subplot(n_rows,5,4+ctr_id*5)
    scale = 20
    plt.plot([x_v1*-scale*2, x_v1*scale*2],
             [y_v1*-scale*2, y_v1*scale*2], color='red')
    plt.plot([x_v2*-scale, x_v2*scale],
             [y_v2*-scale, y_v2*scale], color='blue')
    plt.plot(x, y, 'k.')
    plt.axis('equal')
    plt.gca().invert_yaxis()  # Match the image system with origin at top left

    #fig=plt.figure()

    theta = np.arctan((x_v1)/(y_v1))  
    rotation_mat = np.matrix([[np.cos(theta), -np.sin(theta)],
                          [np.sin(theta), np.cos(theta)]])
    transformed_mat = rotation_mat * coords

    # plot the transformed blob
    x_transformed, y_transformed = transformed_mat.A
    plt.plot(x_transformed, y_transformed, 'g.')


    # ROTATE RAW DATA
    ax=plt.subplot(n_rows, 5,5+ctr_id*5)
    try: 
        frame_rotated = rotate_image(frame_patch, -theta*180/3.14159)
        frame_rotated = frame_rotated[padding:-padding, padding: -padding]
        plt.imshow(frame_rotated,aspect=aspect)
    except:
        pass
plt.show()


IndexError: list index out of range

In [227]:
##################################################################
#################### REMOVE PATCHES BELOW THRESHOLD ##############
##################################################################

centres = []
ids = []
min_size = 0
for k in range(102):
    idx = np.unique(img_array[k])
    centres.append([])
    ids.append([])
    #print (k, idx.shape, idx)
    ctr=0
    for id_ in idx:
        idx2 = np.where(img_array[k]==id_)
        if idx2[0].shape[0]< min_size:
            img_array[k][idx2]=0
            continue
        ids[k].append(ctr)
        if idx2[0].shape[0]>1:
            centres[k].append(np.array([np.mean(idx2[0]), np.mean(idx2[1])]))
        else:
            centres[k].append(np.array([idx2[0], idx2[1]]))
        ctr+=1    
            
        #print (centres[k])
    #print('')

In [230]:
# LINK CENTRES ACROSS TIME
centres = np.array(centres)
print (centres.shape)
from scipy.spatial import distance

matches = []
for k in range(1,102,1):
    
    # find min distances 
    dist = distance.cdist(centres[k],centres[k-1]) 
    print (k, dist.min(axis=1))
    match = dist.argmin(axis=1)
    print ("k, ", k, "  match: ",  match)
    matches.append(match)
    
matches = np.array(matches)
print (matches.shape)

(102,)


ValueError: could not broadcast input array from shape (2,1) into shape (2)

In [216]:
# FIND CENTRES FOR ALL THE CONTOURS
#centres = []
#min_size = 100
imgs = img_array.copy()
fig=plt.figure(figsize=(20,20))
ctr_plot=0
for k in range(20,40,1):
    ax = plt.subplot(4,5,ctr_plot+1)
    #idx = np.unique(img_array[k])
    #print (k, idx.shape, idx)
    ctr=0
    for id_ in ids[k]:
        idx2 = np.where(img_array[k]==id_)
        #if idx2[0].shape[0]< min_size:
        #    continue
        print ("frame: ", k, " animal id: ", id_, " ctr: ", ctr)
        print ("ids ", ids[k])
        print ("matches ", matches[k])
        print ("   matches ", matches[k-1][ctr])
        #print (matches[k])
        imgs[k][idx2]=matches[k-1][ctr]
        ctr+=1
        
    plt.ylabel(str(k))
    idx =np.where(imgs[k]<0)
    imgs[k][idx]=0
    
    vmin = np.min(imgs[k]); vmax=np.max(imgs[k])
    img2 = surfaces[k]
    img2 = (img2-np.min(img2))/(np.max(img2)-np.min(img2))
    img2 = img2*(vmax-vmin)+vmin
    img_final = np.vstack((imgs[k],img2))
    img_final[img_final.shape[0]//2]=np.nan
    plt.imshow(img_final)
    plt.xticks([])
    plt.yticks([])
    ctr_plot+=1
    
#plt.show()
#"/home/cat/watershed.png",dpi=1200)
fig.savefig("/home/cat/watershed.svg", format='svg', dpi=300)
plt.close()

frame:  20  animal id:  0  ctr:  0
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  0
frame:  20  animal id:  1  ctr:  1
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  2
frame:  20  animal id:  2  ctr:  2
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  2
frame:  20  animal id:  3  ctr:  3
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  3
frame:  20  animal id:  4  ctr:  4
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  5
frame:  20  animal id:  5  ctr:  5
ids  [0, 1, 2, 3, 4, 5]
matches  [0 2 2 1 2 3 4 5]
   matches  6
frame:  21  animal id:  0  ctr:  0
ids  [0, 1, 2, 3, 4, 5, 6, 7]
matches  [0 1 3 4 5 5 6 7]
   matches  0
frame:  21  animal id:  1  ctr:  1
ids  [0, 1, 2, 3, 4, 5, 6, 7]
matches  [0 1 3 4 5 5 6 7]
   matches  2
frame:  21  animal id:  2  ctr:  2
ids  [0, 1, 2, 3, 4, 5, 6, 7]
matches  [0 1 3 4 5 5 6 7]
   matches  2
frame:  21  animal id:  3  ctr:  3
ids  [0, 1, 2, 3, 4, 5, 6, 7]
matches  [0 1 3 4

In [176]:
frame = 24
ax =plt.subplot(1,2,1)
plt.imshow(img_array[frame])
ax =plt.subplot(1,2,2)
plt.imshow(surfaces[frame])

plt.show()

In [25]:
from matplotlib import pyplot as plt
from celluloid import Camera

fig = plt.figure()
camera = Camera(fig)
for i in range(100):
    
    plt.imshow(surfaces[i])
    camera.snap()
animation = camera.animate()
animation.save('/home/cat/surfaces.mp4',fps=30)

In [21]:
x_train = np.load('/home/cat/x_train.npy')
print (x_train.shape)

for k in range(10):
    ax=plt.subplot(2,5,k+1)
    print (x_train[k*27])
    plt.imshow(np.int32(x_train[k*27]))
plt.show()

(2000, 34, 34, 3)
[[[  6.   6.   6.]
  [ 17.  17.  17.]
  [ 16.  16.  16.]
  ...
  [103. 103. 103.]
  [ 57.  57.  57.]
  [  5.   5.   5.]]

 [[ 14.  14.  14.]
  [ 17.  17.  17.]
  [ 16.  16.  16.]
  ...
  [ 65.  65.  65.]
  [ 81.  81.  81.]
  [ 32.  32.  32.]]

 [[ 18.  18.  18.]
  [ 20.  20.  20.]
  [ 19.  19.  19.]
  ...
  [ 59.  59.  59.]
  [ 64.  64.  64.]
  [ 21.  21.  21.]]

 ...

 [[ 30.  30.  30.]
  [ 14.  14.  14.]
  [ 14.  14.  14.]
  ...
  [135. 135. 135.]
  [144. 144. 144.]
  [ 43.  43.  43.]]

 [[ 74.  74.  74.]
  [ 66.  66.  66.]
  [ 40.  40.  40.]
  ...
  [145. 145. 145.]
  [130. 130. 130.]
  [  6.   6.   6.]]

 [[  2.   2.   2.]
  [ 32.  32.  32.]
  [ 26.  26.  26.]
  ...
  [ 48.  48.  48.]
  [ 29.  29.  29.]
  [  0.   0.   0.]]]
[[[  6.   6.   6.]
  [ 17.  17.  17.]
  [ 16.  16.  16.]
  ...
  [103. 103. 103.]
  [ 57.  57.  57.]
  [  5.   5.   5.]]

 [[ 14.  14.  14.]
  [ 17.  17.  17.]
  [ 16.  16.  16.]
  ...
  [ 65.  65.  65.]
  [ 81.  81.  81.]
  [ 32.  32.  32.]]

